In [1]:
from lsc_repeats import LSCRepeatSample
from libra_toolbox.tritium.model import ureg, Model, quantity_to_activity
import numpy as np
import json
from libra_toolbox.tritium.lsc_measurements import (
    LIBRARun,
    LSCFileReader,
    GasStream,
    LSCSample,
    LIBRASample,
)
from datetime import datetime

all_file_readers = []
all_quench = []

In [20]:
def create_sample(label: str, filename: str) -> LSCRepeatSample:
    """
    Create a LSCRepeatSample from a LSC file with background substracted.

    Args:
        label: the label of the sample in the LSC file
        filename: the filename of the LSC file

    Returns:
        the LSCRepeatSample object
    """
    # check if a LSCFileReader has been created for this filename
    found = False
    for file_reader in all_file_readers:
        if file_reader.filename == filename:
            found = True
            break

    # if not, create it and add it to the list of LSCFileReaders
    if not found:
        file_reader = LSCFileReader(filename, labels_column="SMPL_ID")

    file_reader.read_file()

    # create the sample
    sample = LSCRepeatSample.from_file(file_reader, label)

   # try to find the background sample from the file
    background_labels = ["1L-BL-1", "1L-BL-2", "1L-BL-3"]
    background_sample = None

    for background_label in background_labels:
        try:
            background_sample = LSCRepeatSample.from_file(file_reader, background_label)
            break
        except ValueError:
            continue

    if background_sample is None:
        raise ValueError(f"Background sample not found in {filename}")

    # substract background
    sample.substract_background(background_sample)

    # read quench set
    all_quench.append(file_reader.quench_set)

    return sample


lsc_data_folder = "../../data/tritium_detection"
with open("../../data/general.json", "r") as f:
    general_data = json.load(f)

run_nb = general_data["general_data"]["run_nb"]


# read start time from general.json
all_start_times = []
for generator in general_data["generators"]:
    if generator["enabled"] is False:
        continue
    for irradiation_period in generator["periods"]:
        start_time = datetime.strptime(irradiation_period["start"], "%m/%d/%Y %H:%M")
        all_start_times.append(start_time)
start_time = min(all_start_times)


In [27]:
stream = "IV"
sample_nb = "2"
vial_nb = "4"
filename = "REPEAT_IV-2-1_TO_2-10.csv"

sample = create_sample(
    label=f"1L-{stream}_{run_nb}-{sample_nb}-{vial_nb}",
    filename=f"{lsc_data_folder}/{filename}",
)

In [23]:
print(sample.name)
print(sample.counted_activities)
print(sample.avg_activity)
print(sample.stdev_activity)
print(sample.background_substracted)
print(sample.rep_number)

1L-IV_2-2-3
[5.234 5.224 5.207 5.177 5.352 5.228] becquerel
4.853999999999999 becquerel
0.06104096984812763 becquerel
True
6


In [ ]:
import pint
from typing import List, Dict
import numpy as np
import warnings

def stdev_addition(stdevs: List[pint.Quantity]) -> pint.Quantity:
    """Calculates stdev of added variables 
    """
    result = np.sqrt(np.sum(std**2 for std in stdevs))
    return result

class LSCRepeatSample:
    name: str | None
    counted_activities: List[pint.Quantity] | None
    rep_number: int | None
    avg_activity: pint.Quantity | None
    stdev_activity: pint.Quantity | None
    origin_file: str | None

    def __init__(self, name: str, counted_activities: List[pint.Quantity]):
        self.name = name
        self.counted_activities = counted_activities
        self.rep_number = len(self.counted_activities)
        self._perform_statistics(self)
        self.background_substracted = False
        self.origin_file = None

    def _perform_statistics(self):
        self.avg_activity = np.mean(self.counted_activities)
        self.stdev_activity = np.std(self.counted_activities, ddof=1) if self.rep_number > 1 else 0.0 

    def __str__(self):
        return f"Sample {self.name}"
    
    def background_substract(self, background_sample: LSCRepeatSample):
        """Substracts the background activity from the sample activity, and updates standard deviation.
        Note: Does NOT substract background from individual counts, only from average

        Args:
            background_sample (LSCRepeatSample): Background sample

        Raises:
            ValueError: If background has already been substracted
        """
        if self.background_substracted:
            raise ValueError("Background already substracted")
        self.avg_activity -= background_sample.avg_activity
        if self.avg_activity.magnitude < 0:
            warnings.warn(
                f"Activity of {self.name} is negative after substracting background. Setting to zero."
            )
            self.activity = 0 * ureg.Bq
        self.stdev_activity = stdev_addition((self.stdev_activity, background_sample.stdev.activity))
        self.background_substracted = True






In [1]:
from collections import defaultdict
from typing import List, DefaultDict

def group_repeats(labels: List[str], values: List[float]) -> DefaultDict[str, List[float]]:
    """
    Groups values by labels and returns a defaultdict where each label maps to a list of values.

    Parameters:
    - labels (List[str]): A list of labels (strings).
    - values (List[float]): A list of corresponding values (floats or ints).

    Returns:
    - defaultdict[str, List[float]]: A dictionary where each label maps to a list of values.

    Raises:
    - ValueError: If labels and values are not lists or have different lengths.
    """

    # Validate inputs
    if not isinstance(labels, list) or not isinstance(values, list):
        raise ValueError("Both labels and values must be lists.")
    
    if len(labels) != len(values):
        raise ValueError("labels and values must have the same length.")
    
    if not all(isinstance(label, str) for label in labels):
        raise ValueError("All elements in labels must be strings.")
    
    if not all(isinstance(value, (int, float)) for value in values):
        raise ValueError("All elements in values must be integers or floats.")

    # Group values by labels
    data: DefaultDict[str, List[float]] = defaultdict(list)
    for label, value in zip(labels, values):
        data[label].append(value)

    return data

In [15]:


filename = "REPEAT_IV-2-1_TO_2-10.csv"
file_reader = LSCFileReader(f"{lsc_data_folder}/{filename}", labels_column="SMPL_ID")

file_reader.read_file()

test_dict = file_reader.get_bq1_values_with_labels()
test_vals = file_reader.get_bq1_values()
test_labels = file_reader.vial_labels



labelled_values = {label: val for label, val in zip(file_reader.vial_labels, test_vals)}
test_zip = list(zip(file_reader.vial_labels, test_vals))
print(test_zip)

[('1L-Bl-3', 0.37), ('1L-Bl-3', 0.38), ('1L-Bl-3', 0.39), ('1L-Bl-3', 0.376), ('1L-Bl-3', 0.38), ('1L-Bl-3', 0.402), (nan, 0.383), (nan, nan), ('1L-IV_2-1-3', 2.293), ('1L-IV_2-1-3', 2.327), ('1L-IV_2-1-3', 2.23), ('1L-IV_2-1-3', 2.286), ('1L-IV_2-1-3', 2.259), ('1L-IV_2-1-3', 2.289), (nan, 2.281), ('1L-IV_2-1-4', 0.411), ('1L-IV_2-1-4', 0.419), ('1L-IV_2-1-4', 0.396), ('1L-IV_2-1-4', 0.403), ('1L-IV_2-1-4', 0.408), ('1L-IV_2-1-4', 0.397), (nan, 0.406), (nan, nan), ('1L-IV_2-2-3', 5.234), ('1L-IV_2-2-3', 5.224), ('1L-IV_2-2-3', 5.207), ('1L-IV_2-2-3', 5.177), ('1L-IV_2-2-3', 5.352), ('1L-IV_2-2-3', 5.228), (nan, 5.237), ('1L-IV_2-2-4', 0.541), ('1L-IV_2-2-4', 0.549), ('1L-IV_2-2-4', 0.554), ('1L-IV_2-2-4', 0.554), ('1L-IV_2-2-4', 0.523), ('1L-IV_2-2-4', 0.547), (nan, 0.545), (nan, nan), ('1L-IV_2-3-3', 10.588), ('1L-IV_2-3-3', 10.633), ('1L-IV_2-3-3', 10.727), ('1L-IV_2-3-3', 10.602), ('1L-IV_2-3-3', 10.499), ('1L-IV_2-3-3', 10.527), (nan, 10.596), ('1L-IV_2-3-4', 1.4), ('1L-IV_2-3-4',

In [ ]:
# create gas streams
gas_streams = {}
for stream, samples in general_data["tritium_detection"].items():
    stream_samples = []
    for sample_nb, sample_dict in samples.items():
        libra_samples = []
        if sample_dict["actual_sample_time"] is None:
            continue
        for vial_nb, filename in sample_dict["lsc_vials_filenames"].items():
            sample = create_sample(
                label=f"1L-{stream}_{run_nb}-{sample_nb}-{vial_nb}",
                filename=f"{lsc_data_folder}/{filename}",
            )
            libra_samples.append(sample)

        time_sample = datetime.strptime(
            sample_dict["actual_sample_time"], "%m/%d/%Y %H:%M"
        )
        stream_samples.append(LIBRASample(libra_samples, time=time_sample))
    gas_streams[stream] = GasStream(stream_samples, start_time=start_time)